In [13]:
import requests
import pandas as pd
import folium 
from folium.plugins import MarkerCluster, MiniMap

In [16]:
API_KEY = '8b50343fe07230c95dea834f1b4e9559'

In [75]:
def add2latlng(address):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    header = {'Authorization': 'KakaoAK ' + API_KEY}
 
    r = requests.get(url, headers=header)
 
    if r.status_code == 200:
        documents = r.json()["documents"]
        if len(documents) > 0:
            result_address = r.json()["documents"][0]["address"]
            return result_address["y"], result_address["x"]
        else:
            print ("No data:", address)
            return None
    else:
        print ("ERROR[" + str(r.status_code) + "]")
        return None


# 주소 데이터 로드

In [77]:
def get_lat_lng(df):
    df['latitude'] = 0.0
    df['longitude'] = 0.0
    
    for i, row in enumerate(df.iterrows()):
        lat_lng = add2latlng(row[1]['주소'])
    
        if lat_lng:
            df.at[i, 'latitude'] = lat_lng[0]
            df.at[i, 'longitude'] = lat_lng[1]

In [74]:
df_baskin = pd.read_csv('data/베스킨라빈스.csv')

In [79]:
get_lat_lng(df_baskin)

No data: 서울 서초구 강남대로 지하 390
No data: 서울 서초구 방배3동 1001-31
No data: 서울 서초구 태봉로 70, 107-1호
No data: 서울 강남구 자곡동 383
No data: 서울 용산구 효창동 13-2
No data: 서울 성동구 홍익동 550 센트라스 상가 L동 B19호
No data: 서울 성북구 인촌로24길 25 (안암동5가)
No data: 서울 영등포구 당산로 214 (당산동5가, 당산 삼성 래미안)111-2, 111-3호
No data: 서울 구로구 신도림동 436-7 큐븨스테이트 오피스텔 상가 1층
No data: 서울 강북구 도봉로 8길 5
No data: 서울 강동구 고덕동 212 고덕그라시움아파트 상가4동 1층
No data: 서울 강동구 아리수로 427 106호


In [120]:
df_baskin = df_baskin[df_baskin['latitude'] > 0.0]

In [80]:
df_mega = pd.read_csv('data/메가커피.csv')

In [82]:
get_lat_lng(df_mega)

No data: 서울특별시 송파구 법원로11길 25, 108호(문정동)
No data: 서울특별시 마포구 월드컵북로54길 17130호(상암동 1597)
No data: 서울특별시 송파구 법원로4길 6, 132-2호(문정동)
No data: 서울특별시 송파구 위례광장로 270, 120호(장지동, 송파와이즈더샵)
No data: 서울특별시 강동구 성내로 11, 105호(성내동)
No data: 서울특별시 강남구 도곡로77길 6, 102호(대치동)
No data: 서울특별시 강남구 삼성로 155, 116호(대치동)
No data: 서울특별시 강동구 천호대로 997, 107호(천호동)
No data: 서울특별시 강남구 도곡로 405, 105호(대치동)
No data: 서울특별시 서대문구 연희로 98 1-2층(연희동)


In [122]:
df_mega = df_mega[df_mega['latitude'] > 0.0]

In [123]:
len(df_mega)

247

# 지도 생성

In [97]:
def add_markers(df, color, name):
    for i, row in enumerate(df.iterrows()):
        latitude = row[1]['latitude']
        longitude = row[1]['longitude']

        if latitude > 0.0 and longitude > 0.0:
            #marker_cluster.add_child(folium.Marker(location=[latitude, longitude], popup='베스킨라빈스')) 
            folium.Marker(location=[latitude, longitude], popup=name, icon=folium.Icon(color=color)).add_to(map_hs)

In [98]:
map_hs = folium.Map((37.5498849819149, 126.874971203942), zoom_start = 13) 
# marker_cluster = MarkerCluster()

add_markers(df_baskin, 'pink', '베스킨')
add_markers(df_mega, 'gray', '메가')

In [103]:
map_hs

In [100]:
map_hs.save("베스킨_메가_비교.html")

In [108]:
from geopy.distance import geodesic

In [132]:
df_baskin['closest_mega'] = 100000

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [130]:
def get_min_distance(latitude, longitude):
    min_dist = 1000000
    
    for row in df_mega.iterrows():
        dist = geodesic((latitude, longitude), (row[1]['latitude'], row[1]['longitude'])).meters
        if dist < min_dist:
            min_dist = dist
    
    return min_dist

In [133]:
for i, row in df_baskin.iterrows():
    latitude = row[1]
    longitude = row[2]
    
    if latitude > 0.0 and longitude > 0.0:
        min_dist = get_min_distance(latitude, longitude)
        
        df_baskin.at[i, 'closest_mega'] = min_dist

In [135]:
df_baskin_no_mega = df_baskin[df_baskin['closest_mega'] > 500]

In [136]:
len(df_baskin_no_mega)

101

In [137]:
df_baskin_no_mega

,주소,latitude,longitude,주변_메가_있음,closest_mega
3,서울 서초구 서초동 1337-34 1층,37.492736,127.029663,True,668
4,"서울 서초구 서초대로 331 (서초동, 정우빌딩)",37.495149,127.017636,True,533
6,서울 서초구 고무래로8길 2,37.502775,127.010521,True,616
7,"서울 서초구 남부순환로 지하 2585 (서초동, 양재역)",37.483935,127.034544,True,1727
8,서울 서초구 남부순환로 2620 지하1층,37.484514,127.037328,True,1767
...,...,...,...,...,...
283,"서울 구로구 서해안로 2096, 1층 105호, 106호 (항동)",37.479763,126.822891,True,1393
287,서울 은평구 진관동 19 상림마을 820동 110호,37.644197,126.922772,True,608
291,서울 노원구 동일로 1541,37.665674,127.057050,True,850
292,서울 노원구 덕릉로118길 7 (상계동),37.667575,127.079310,True,931


In [138]:
for row in df_baskin_no_mega.iterrows():
    folium.Circle([row[1]['latitude'], row[1]['longitude']], radius=500).add_to(map_hs)

In [139]:
map_hs

In [140]:
map_hs.save("베스킨_메가_비교.html")